In [8]:
import time

start_time = time.time()

def parse_data_line(s):
    addr = int(s[:5])-40001
    com_obj = int(s[17:26])
    name = s[26:41].strip()
    dim = s[41:46].strip()
    data_type = s[46:58].strip()
    data_len = int(s[58:60].strip())
    dec = s[60:63].strip()
    data_min = s[63:70].strip()
    data_max = s[70:77].strip()
    group = s[77:].strip()
    return addr, com_obj, name, dim, data_type, data_len, dec, data_min, data_max, group

def read_mb_register(i, reg):
    for i in range(10):
        try:
            return i.read_register(reg, signed=True)
        except Exception as e:
            print(e)
            return None
                
def read_mb_registers_long(g, reg):
    for i in doc:
        while 1:
            try:
                if i.address == 1 or i.address == 2:
                    print(str(i.read_long(reg)))
                elif i.address == 3 or i.address == 4 or i.address == 5:
                    print(i.read_long(reg))
                break
            except:
                continue

with open('./mb_table/doc_3520_mb.txt', encoding='cp1252') as f:
    data_lines = f.readlines()
    
data_dict = {}
count = 0

for i in data_lines[2:]:
    if i[0] == '\n': # Если пустая строка, то заканчиваем опрос
        break
    elif i[0] == '4': # Если начинается с 4-х, то
        line = parse_data_line(i)
        if line[5]<3:
            count += 1
            data = read_mb_register(g3, line[0])
            if line[6] != '-' and line[6] != '0' and line[6] != None:
                data = data/10**int(line[6])
            print(f'{count} {line[2]}: \t {data} {line[3]}')

# for addr in data_arr:
#     count += 1
#     if isinstance(addr, int):
#         print(addr, read_mb_register(g1, addr))
#     else:
#         pass
#         if int(addr.strip()[-2]) != 2:
#             a = int(addr.split('-')[0] - 40001)
#             print(addr, 'long', '\t', read_mb_registers_long(g1, addr))
        
print((f'{time.time() - start_time:.1f} seconds'))

'int' object has no attribute 'read_register'
1 BIN: 	 None 
'int' object has no attribute 'read_register'
2 BIN-2: 	 None 
'int' object has no attribute 'read_register'
3 BIN-5: 	 None 
'int' object has no attribute 'read_register'
4 BIN-6: 	 None 
'int' object has no attribute 'read_register'
5 BIN-7: 	 None 
'int' object has no attribute 'read_register'
6 BIN-8: 	 None 
'int' object has no attribute 'read_register'
7 BIN-9: 	 None 
'int' object has no attribute 'read_register'
8 BIN-10: 	 None 
'int' object has no attribute 'read_register'
9 BIN-11: 	 None 
'int' object has no attribute 'read_register'
10 BIN-12: 	 None 
'int' object has no attribute 'read_register'
11 BOUT: 	 None 
'int' object has no attribute 'read_register'


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [70]:
start_time = time.time()

def read_mb_register(i, reg):
    while 1:
        try:
            return i.read_register(reg)
        except Exception as e:
            continue
            
def get_protect(genset):
    level = ['inactive', 'N/A', 'active, confirmed', 'active, but blocked or delay still running', 'previously active, not confirmed yet', 'N/A','active, not confirmed yet', 'active, not confirmed yet, blocked']
    sensfail = ['Sensor failure not active', 'Sensor failure active, confirmed', 'Sensor failure previously active, not confirmed yet', 'Sensor failure active, not confirmed yet']
    
    if genset.address == 1 or genset.address == 2:
        with open('protections_3516.txt') as f:
            data_lines = f.readlines()
    elif genset.address == 3 or genset.address == 4 or genset.address == 5:
        with open('protections_3520.txt') as f:
            data_lines = f.readlines()
        
    protections = []
    ret = []
    
    for line in data_lines:
        # [6211, 'Bus V L2-L3', 'Bus V L3-L1']
        protections.append((int(line[:5])-40001, line[17:37].strip(), line[37:].strip()))
        
    for p in protections:
        addr, protect2, protect1 = p
        protection = read_mb_register(genset, addr)
    
        prot1_level1 = 0b0000000000000111 & protection
        prot1_level2 = (0b0000000000111000 & protection)>>3
        prot1_sens = (0b0000000011000000 & protection)>>6
        prot2_level1 = 0b0000011100000000 & protection>>8
        prot2_level2 = (0b0011100000000000 & protection)>>11
        prot2_sens = (0b1100000000000000 & protection)>>14

        if prot1_level1: ret.append(f'{protect1}. Level 1: {level[prot1_level1]}')
        if prot1_level2: ret.append(f'{protect1}. Level 2: {level[prot1_level2]}')
        if prot1_sens: ret.append(f'{protect1}. Sensor failure: {sensfail[prot1_sens]}')
        if prot2_level1: ret.append(f'{protect2}. Level 1: {level[prot2_level1]}')
        if prot2_level2: ret.append(f'{protect2}. Level 2: {level[prot2_level2]}')
        if prot2_sens: ret.append(f'{protect2}. Sensor failure: {sensfail[prot2_sens]}')
        
    return ret

for i in get_protect(g5):
    print(i)
    
print((f'{time.time() - start_time:.1f} seconds'))

KeyboardInterrupt: 

In [9]:
import telebot
import minimalmodbus 

g1 = minimalmodbus.Instrument('/dev/ttyS0', 1)
g1.serial.baudrate = 19200

token = '1325955552:AAHQJzvhu-IhcMpsVGX5dUsfXTJ7fFECoJs'
tb = telebot.TeleBot(token)

tb.send_message(723253749, f'Наработка ГПГУ 1: {g1.read_register(262)}')

'QF17,18,19 OFF. Level 1: active, confirmed\nOil Pressure. Level 1: active, but blocked or delay still running\n'

In [42]:
import telebot
import minimalmodbus 
import time
import logging

# logging.basicConfig(format='%(asctime)s, %(levelname)s, %(message)s', filename='debug_log', level=logging.DEBUG)
logging.basicConfig(format='%(asctime)s, %(levelname)s, %(message)s', level=logging.DEBUG)

g1 = minimalmodbus.Instrument('/dev/ttyS0', 1)
g1.serial.baudrate = 19200

g2 = minimalmodbus.Instrument('/dev/ttyS0', 2)
g2.serial.baudrate = 19200

g3 = minimalmodbus.Instrument('/dev/ttyS0', 3)
g3.serial.baudrate = 19200

g4 = minimalmodbus.Instrument('/dev/ttyS0', 4)
g4.serial.baudrate = 19200

g5 = minimalmodbus.Instrument('/dev/ttyS0', 5)
g5.serial.baudrate = 19200

token = '1325955552:AAHQJzvhu-IhcMpsVGX5dUsfXTJ7fFECoJs'
tb = telebot.TeleBot(token)

gensets = ('ГПГУ 1', 'ГПГУ 2', 'ГПГУ 3', 'ГПГУ 4', 'ГПГУ 5')
doc = (g1, g2, g3, g4, g5)

def read_mb_register(i, reg):
    for c in range(10):
        try:
            return i.read_register(reg)
        except Exception as e:
            continue
    return -1

def get_engines_state():
    logging.debug('get_engines_state()')
    engine_state_3516 = ('Init', 'Ready', 'NotReady', 'Prestart', 'Cranking', 'Pause', 'Starting', 'Running', 'Loaded', 'Soft unld', 'Cooling', 'Stop', 'Shutdown', 'Ventil', 'EmergMan', 'Cooldown', 'Offload', 'Soft load', 'WaitStop', 'Warming', 'SDVentil', None)
    engine_state_3520 = ('Init', 'Ready', 'NotReady', 'Prestart', 'Cranking', 'Pause', 'Starting', 'Running', 'Loaded', 'Soft unld', 'Cooling', 'Stop', 'Shutdown', 'Ventil', 'EmergMan', 'Cooldown', 'Offload', 'Soft load', 'WaitStop', 'Warming', 'SDVentil', 'WD test', 'GasVTest', 'StrtCndWai', None)
    ret = [engine_state_3516[read_mb_register(g1, 162)], engine_state_3516[read_mb_register(g2, 162)], engine_state_3520[read_mb_register(g3, 295)], engine_state_3520[read_mb_register(g4, 295)], engine_state_3520[read_mb_register(g5, 295)]]
    logging.debug(ret)
    return ret

def get_breakers_state():
    logging.debug('get_breakers_state()')
    breaker_state = ('Init', 'BrksOff', 'IslOper', 'MainsOper', 'ParalOper', 'RevSync', 'Synchro', 'MainsFlt', 'ValidFlt', 'MainsRet', 'MultIslOp', 'MultParOp', 'EmergMan', 'StrUpSync', None)
    ret = [breaker_state[read_mb_register(g1, 163)], breaker_state[read_mb_register(g2, 163)], breaker_state[read_mb_register(g3, 296)], breaker_state[read_mb_register(g4, 296)], breaker_state[read_mb_register(g5, 296)]]
    logging.debug(ret)
    return ret

def get_protect(genset):
    logging.debug(f'get_protect(genset {genset.address})')
    level = ['inactive', 'N/A', 'active, confirmed', 'active, but blocked or delay still running', 'previously active, not confirmed yet', 'N/A','active, not confirmed yet', 'active, not confirmed yet, blocked', None]
    sensfail = ['Sensor failure not active', 'Sensor failure active, confirmed', 'Sensor failure previously active, not confirmed yet', 'Sensor failure active, not confirmed yet', None]
    
    if genset.address in (1, 2):
        with open('protections_3516.txt') as f:
            data_lines = f.readlines()
    elif genset.address in (3, 4, 5):
        with open('protections_3520.txt') as f:
            data_lines = f.readlines()
        
    protections = []
    ret = []
    
    for line in data_lines:
        # [6211, 'Bus V L2-L3', 'Bus V L3-L1']
        protections.append((int(line[:5])-40001, line[17:37].strip(), line[37:].strip()))
        
    for p in protections:
        addr, protect2, protect1 = p
        protection = read_mb_register(genset, addr)
        if protection == -1: continue
    
        prot1_level1 = 0b0000000000000111 & protection
        prot1_level2 = (0b0000000000111000 & protection)>>3
        prot1_sens = (0b0000000011000000 & protection)>>6
        prot2_level1 = 0b0000011100000000 & protection>>8
        prot2_level2 = (0b0011100000000000 & protection)>>11
        prot2_sens = (0b1100000000000000 & protection)>>14
        
        if prot1_level1: ret.append(f'{protect1}. Level 1: {level[prot1_level1]}\n')
        if prot1_level2: ret.append(f'{protect1}. Level 2: {level[prot1_level2]}\n')
        if prot1_sens: ret.append(f'{protect1}. Sensor failure: {sensfail[prot1_sens]}\n')
        if prot2_level1: ret.append(f'{protect2}. Level 1: {level[prot2_level1]}\n')
        if prot2_level2: ret.append(f'{protect2}. Level 2: {level[prot2_level2]}\n')
        if prot2_sens: ret.append(f'{protect2}. Sensor failure: {sensfail[prot2_sens]}\n')
    
    logging.debug(ret)
        
    return ret

def get_gcb_state():
    logging.debug('get_gcb_state()')
    g1_bin1, g2_bin1, g3_bin2, g4_bin2, g5_bin2 = read_mb_register(g1, 2), read_mb_register(g2, 2), read_mb_register(g3, 7), read_mb_register(g4, 7), read_mb_register(g5, 7) 
    ret = [(1<<2&g1_bin1)>>2, (1<<2&g2_bin1)>>2, 1&g3_bin2, 1&g4_bin2, 1&g5_bin2]
    logging.debug(ret)
    return ret

es_old = get_engines_state()
bs_old = get_breakers_state()
gs_old = get_gcb_state()
prtct_old = [get_protect(g) for g in doc]

# Основной цикл
while 1:
    
#     start_time = time.time()

    es_new = get_engines_state()
    bs_new = get_breakers_state()
    gs_new = get_gcb_state()
    
#     print((f'{time.time() - start_time:.1f} seconds'))
    
    for i in range(5):
        # Читаю состояния двигателей
        if es_new[i] == None: es_new[i] = es_old[i] 
        if es_new[i] != es_old[i]:
            es_old[i] = es_new[i]
            if es_new[i] in ('Init', 'Ready', 'NotReady', 'Starting', 'Soft unld', 'Stop', 'Shutdown', 'Ventil', 'EmergMan', 'Cooldown', 'Offload', 'WaitStop', 'Warming', 'SDVentil'):
                tb.send_message(723253749, f'{gensets[i]}. Engine state: {es_new[i]}')
            if es_new[i] in ('NotReady', 'Shutdown', 'Soft unld'):
                msg = ''
                for i in get_protect(doc[i]):
                    msg += i
                if msg: tb.send_message(723253749, msg)
        # Читаю статусы выключателей
        if bs_new[i] == None: bs_new[i] = bs_old[i] 
        if bs_new[i] != bs_old[i]:
            bs_old[i] = bs_new[i]
            if bs_new[i] in ('Init', 'BrksOff', 'IslOper', 'ParalOper', 'RevSync', 'MainsFlt', 'ValidFlt', 'MainsRet', 'MultIslOp', 'EmergMan', 'StrUpSync'):
                tb.send_message(723253749, f'{gensets[i]}. Breaker state: {bs_new[i]}')
        # Читаю состояния выключателей
        if gs_new[i] != gs_old[i]:
            gs_old[i] = gs_new[i]
            if gs_new[i]:
                tb.send_message(723253749, f'{gensets[i]}. GCB Close')
            else:
                tb.send_message(723253749, f'{gensets[i]}. GCB Open')

    time.sleep(1)

KeyboardInterrupt: 

In [36]:
b2 = read_mb_register(g5, 7)

print(bin(b2))

bin2 = [
'GCB Feedback',
'Reserve',
'GCB Alarm',
'FaultResButton',
'SA102Auto Mode',
'SA102Test Mode',
'AC Pump Auto',
'AC Pump Manual'
]

mask = 1

for i in range(8):
    flag = 0
    if b2 & mask: flag = 1
    print(f'{bin2[i]} \t {flag}')
    mask = mask << 1

1&b2

-0b1
GCB Feedback 	 1
Reserve 	 1
GCB Alarm 	 1
FaultResButton 	 1
SA102Auto Mode 	 1
SA102Test Mode 	 1
AC Pump Auto 	 1
AC Pump Manual 	 1


1

In [55]:
b1 = g2.read_register(2)

print(bin(b1))

bin1 = ['RemStart/Stop',
'Emergency stop',
'GCB Closed',
'GCB Open',
'GCB Fail',
'GenMess QF2SHS',
'BusMess QF3SHS',
'FireAlarm',
'CO/CH4Lev1',
'CO/CH4Lev2',
'RFan M1-M3 On',
'RFan M1 Fault',
'RFan M2 Fault',
'QF17,18,19 OFF',
'not used',
'RFan M3 Fault']

mask = 1

for i in range(16):
    flag = 0
    if b1 & mask: flag = 1
    print(f'{bin1[i]} \t {flag}')
    mask = mask << 1

(1<<2&b1)>>2

NoResponseError: No communication with the instrument (no answer)

In [8]:
import minimalmodbus, time

# g1 = minimalmodbus.Instrument('/dev/ttyS0', 1)
# g1.serial.baudrate = 19200

# g2 = minimalmodbus.Instrument('/dev/ttyS0', 2)
# g2.serial.baudrate = 19200

# g3 = minimalmodbus.Instrument('/dev/ttyS0', 3)
# g3.serial.baudrate = 19200

# g4 = minimalmodbus.Instrument('/dev/ttyS0', 4)
# g4.serial.baudrate = 19200

# g5 = minimalmodbus.Instrument('/dev/ttyS0', 5)
# g5.serial.baudrate = 19200

# doc = [g1, g2, g3, g4, g5]

class Genset(minimalmodbus.Instrument):
    pass

g1 = Genset?

In [ ]:
g1 = Genset

In [7]:
dir(g1.serial)

['BAUDRATES',
 'BAUDRATE_CONSTANTS',
 'BYTESIZES',
 'PARITIES',
 'STOPBITS',
 '_SAVED_SETTINGS',
 '__abstractmethods__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_baudrate',
 '_break_state',
 '_bytesize',
 '_checkClosed',
 '_checkReadable',
 '_checkSeekable',
 '_checkWritable',
 '_dsrdtr',
 '_dtr_state',
 '_inter_byte_timeout',
 '_parity',
 '_port',
 '_reconfigure_port',
 '_rs485_mode',
 '_rts_state',
 '_rtscts',
 '_set_rs485_mode',
 '_set_special_baudrate',
 '_stopbits',
 '_timeout',
 '_update_break_state',
 '_update_dtr_state',
 '_update_rts_state',
 '_write_timeout',
 '_xonxoff',


In [3]:
def get_mh():
    ret = []
    for g in doc:
        adr = 3586 if g.address in (1, 2) else 3821
        print(adr)
        ret.append(read_mb_registers_long(g, adr))
    return ret
get_mh()

3586


NameError: name 'read_mb_registers_long' is not defined